##Linear Regression##

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
import pickle

# Seed initialization
np.random.seed(123456789)
torch.manual_seed(12345678)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Dataset and parameters
data_path = '../dataset'
market_name = 'NASDAQ'
relation_name = 'wikidata'
stock_num = 1026
lookback_length = 16
epochs = 15
valid_index = 756
test_index = 1008
fea_num = 5
market_num = 20
steps = 1
learning_rate = 0.001
alpha = 0.1
scale_factor = 3

dataset_path = os.path.join('../dataset', market_name)
if market_name == "SP500":
    data = np.load(os.path.join(dataset_path, 'SP500.npy'))
    data = data[:, 915:, :]
    price_data = data[:, :, -1]
    mask_data = np.ones((data.shape[0], data.shape[1]))
    eod_data = data
    gt_data = np.zeros((data.shape[0], data.shape[1]))
    for ticket in range(data.shape[0]):
        for row in range(1, data.shape[1]):
            gt_data[ticket][row] = (data[ticket][row][-1] - data[ticket][row - steps][-1]) / data[ticket][row - steps][-1]
else:
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/eod_data.pkl'), 'rb') as f:
        eod_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/mask_data.pkl'), 'rb') as f:
        mask_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/gt_data.pkl'), 'rb') as f:
        gt_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/price_data.pkl'), 'rb') as f:
        price_data = pickle.load(f)

# Define the LinearRegressionModel (add this if you don't already have it)
class LinearRegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

# Model definition
input_dim = lookback_length * fea_num
output_dim = 1
model = LinearRegressionModel(input_dim, output_dim).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

batch_offsets = np.arange(0, valid_index, dtype=int)

def prepare_input_data(offset, lookback_length, steps):
    seq_len = lookback_length
    mask_batch = mask_data[:, offset: offset + seq_len + steps]
    mask_batch = np.min(mask_batch, axis=1)
    return (
        eod_data[:, offset:offset + seq_len, :].reshape(-1, input_dim),
        np.expand_dims(mask_batch, axis=1),
        np.expand_dims(price_data[:, offset + seq_len - 1], axis=1),
        np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis=1)
    )

def get_batch(offset=None):
    if offset is None:
        offset = np.random.randint(0, valid_index)
    data, mask, price, gt = prepare_input_data(offset, lookback_length, steps)
    return torch.tensor(data, dtype=torch.float32).to(device), torch.tensor(mask, dtype=torch.float32).to(device), torch.tensor(price, dtype=torch.float32).to(device), torch.tensor(gt, dtype=torch.float32).to(device)

def mean_squared_error(y_true, y_pred):
    return torch.mean((y_true - y_pred) ** 2)

def information_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return torch.corrcoef(torch.stack([y_true, y_pred]))[0, 1]

def rank_information_coefficient(y_true, y_pred):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()
    return torch.corrcoef(torch.stack([torch.argsort(y_true), torch.argsort(y_pred)]))[0, 1]

def precision_at_k(y_true, y_pred, k=10):
    _, topk_indices = torch.topk(y_pred, k, dim=0)
    precision = torch.sum(y_true[topk_indices]) / k
    return precision

def sharpe_ratio(y_true, y_pred):
    excess_returns = y_true - y_pred
    return torch.mean(excess_returns) / torch.std(excess_returns)

def validate(start_index, end_index):
    model.eval()
    total_loss = 0
    total_ic = 0
    total_ric = 0
    total_prec_10 = 0
    total_sharpe5 = 0
    count = 0
    with torch.no_grad():
        for offset in range(start_index, end_index):
            data, mask, price, gt = get_batch(offset)
            prediction = model(data)
            loss = criterion(prediction * mask, gt * mask)
            total_loss += loss.item()
            total_ic += information_coefficient(gt * mask, prediction * mask).item()
            total_ric += rank_information_coefficient(gt * mask, prediction * mask).item()
            total_prec_10 += precision_at_k(gt * mask, prediction * mask, k=10).item()
            total_sharpe5 += sharpe_ratio(gt * mask, prediction * mask).item()
            count += 1
    avg_loss = total_loss / count
    avg_ic = total_ic / count
    avg_ric = total_ric / count
    avg_prec_10 = total_prec_10 / count
    avg_sharpe5 = total_sharpe5 / count
    return {
        'mse': avg_loss,
        'IC': avg_ic,
        'RIC': avg_ric,
        'prec_10': avg_prec_10,
        'sharpe5': avg_sharpe5
    }

# Training loop
best_valid_loss = float('inf')
best_valid_perf = None
best_test_perf = None

for epoch in range(epochs):
    model.train()
    np.random.shuffle(batch_offsets)
    train_loss = 0.0
    for j in range(valid_index - lookback_length - steps):
        data, mask, price, gt = get_batch(batch_offsets[j])
        optimizer.zero_grad()
        prediction = model(data)
        loss = criterion(prediction * mask, gt * mask)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= (valid_index - lookback_length - steps)
    print(f'Epoch {epoch + 1}, Training Loss: {train_loss:.4f}')

    val_perf = validate(valid_index, test_index)
    print('Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
        val_perf['mse'], val_perf['IC'], val_perf['RIC'], val_perf['prec_10'], val_perf['sharpe5']))

    test_perf = validate(test_index, len(mask_data))
    print('Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
        test_perf['mse'], test_perf['IC'], test_perf['RIC'], test_perf['prec_10'], test_perf['sharpe5']), '\n\n')

    if val_perf['mse'] < best_valid_loss:
        best_valid_loss = val_perf['mse']
        best_valid_perf = val_perf
        best_test_perf = test_perf

print('Best Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_valid_perf['mse'], best_valid_perf['IC'], best_valid_perf['RIC'], best_valid_perf['prec_10'], best_valid_perf['sharpe5']))
print('Best Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_test_perf['mse'], best_test_perf['IC'], best_test_perf['RIC'], best_test_perf['prec_10'], best_test_perf['sharpe5']))


Epoch 1, Training Loss: 0.0041
Valid performance:
 mse:6.69e-04, IC:-6.34e-03, RIC:7.03e-04, prec@10:6.58e-04, SR:-6.45e-01
Test performance:
 mse:5.70e-04, IC:-4.54e-03, RIC:4.39e-03, prec@10:3.06e-04, SR:-6.88e-01 


Epoch 2, Training Loss: 0.0005
Valid performance:
 mse:5.81e-04, IC:-3.18e-03, RIC:-2.80e-03, prec@10:7.88e-04, SR:-4.66e-01
Test performance:
 mse:4.80e-04, IC:-2.37e-03, RIC:-5.09e-03, prec@10:1.57e-04, SR:-4.92e-01 


Epoch 3, Training Loss: 0.0005
Valid performance:
 mse:5.06e-04, IC:5.23e-03, RIC:4.35e-03, prec@10:1.21e-03, SR:-2.00e-01
Test performance:
 mse:4.05e-04, IC:3.78e-03, RIC:-1.09e-02, prec@10:2.80e-04, SR:-2.00e-01 


Epoch 4, Training Loss: 0.0006
Valid performance:
 mse:5.99e-04, IC:3.07e-02, RIC:-3.25e-04, prec@10:1.96e-03, SR:5.74e-01
Test performance:
 mse:5.58e-04, IC:3.00e-02, RIC:-1.65e-02, prec@10:4.18e-03, SR:6.76e-01 


Epoch 5, Training Loss: 0.0006
Valid performance:
 mse:5.44e-04, IC:3.07e-02, RIC:-1.62e-03, prec@10:1.90e-03, SR:4.38e-01
Te

##KNN##

In [11]:
import random
import numpy as np
import os
import torch as torch
from load_data import load_EOD_data
from evaluator import evaluate
from model import get_loss, StockMixer, KNNStockMixer
import pickle



np.random.seed(123456789)
torch.random.manual_seed(12345678)
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'

# Load data
data_path = '../dataset'
market_name = 'NASDAQ'
relation_name = 'wikidata'
stock_num = 1026
lookback_length = 16
epochs = 15
valid_index = 756
test_index = 1008
fea_num = 5
market_num = 20
steps = 1
learning_rate = 0.001
alpha = 0.1
scale_factor = 3
activation = 'GELU'

dataset_path = '../dataset/' + market_name
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/eod_data.pkl'), 'rb') as f:
        eod_data = pickle.load(f)
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/mask_data.pkl'), 'rb') as f:
    mask_data = pickle.load(f)
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/gt_data.pkl'), 'rb') as f:
    gt_data = pickle.load(f)
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/price_data.pkl'), 'rb') as f:
    price_data = pickle.load(f)

trade_dates = mask_data.shape[1]
model = KNNStockMixer(n_neighbors=5)

def get_batch(offset=None):
    if offset is None:
        offset = random.randrange(0, valid_index)
    seq_len = lookback_length
    mask_batch = mask_data[:, offset: offset + seq_len + steps]
    mask_batch = np.min(mask_batch, axis=1)
    return (
        eod_data[:, offset:offset + seq_len, :],
        np.expand_dims(mask_batch, axis=1),
        np.expand_dims(price_data[:, offset + seq_len - 1], axis=1),
        np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis=1))

def validate(start_index, end_index):
    with torch.no_grad():
        cur_valid_pred = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_gt = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_mask = np.zeros([stock_num, end_index - start_index], dtype=float)
        loss = 0.
        reg_loss = 0.
        rank_loss = 0.
        for cur_offset in range(start_index - lookback_length - steps + 1, end_index - lookback_length - steps + 1):
            data_batch, mask_batch, price_batch, gt_batch = map(
                lambda x: torch.Tensor(x).to(device),
                get_batch(cur_offset)
            )
            prediction = model.predict(data_batch)
            cur_loss, cur_reg_loss, cur_rank_loss, cur_rr = get_loss(prediction, gt_batch, price_batch, mask_batch,
                                                                     stock_num, alpha)
            loss += cur_loss.item()
            reg_loss += cur_reg_loss.item()
            rank_loss += cur_rank_loss.item()
            cur_valid_pred[:, cur_offset - (start_index - lookback_length - steps + 1)] = cur_rr[:, 0].cpu()
            cur_valid_gt[:, cur_offset - (start_index - lookback_length - steps + 1)] = gt_batch[:, 0].cpu()
            cur_valid_mask[:, cur_offset - (start_index - lookback_length - steps + 1)] = mask_batch[:, 0].cpu()
        loss = loss / (end_index - start_index)
        reg_loss = reg_loss / (end_index - start_index)
        rank_loss = rank_loss / (end_index - start_index)
        cur_valid_perf = evaluate(cur_valid_pred, cur_valid_gt, cur_valid_mask)
    return loss, reg_loss, rank_loss, cur_valid_perf

# Training
best_valid_loss = float('inf')
best_valid_perf = None
best_test_perf = None

for epoch in range(epochs):
    print("epoch{}##########################################################".format(epoch + 1))
    batch_offsets = np.arange(start=0, stop=valid_index, dtype=int)
    np.random.shuffle(batch_offsets)
    tra_loss = 0.0
    tra_reg_loss = 0.0
    tra_rank_loss = 0.0

    # Prepare the training data for fitting KNN
    train_data = []
    train_gt = []

    for j in range(valid_index - lookback_length - steps + 1):
        data_batch, mask_batch, price_batch, gt_batch = map(
            lambda x: torch.Tensor(x).to(device),
            get_batch(batch_offsets[j])
        )
        train_data.append(data_batch)
        train_gt.append(gt_batch)

    train_data = torch.cat(train_data, dim=0)
    train_gt = torch.cat(train_gt, dim=0)

    model.fit(train_data, train_gt)

    val_loss, val_reg_loss, val_rank_loss, val_perf = validate(valid_index, test_index)
    print('Valid : loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(val_loss, val_reg_loss, val_rank_loss))

    test_loss, test_reg_loss, test_rank_loss, test_perf = validate(test_index, trade_dates)
    print('Test: loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(test_loss, test_reg_loss, test_rank_loss))

    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        best_valid_perf = val_perf
        best_test_perf = test_perf

    print('Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(val_perf['mse'], val_perf['IC'],
                                                     val_perf['RIC'], val_perf['prec_10'], val_perf['sharpe5']))
    print('Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(test_perf['mse'], test_perf['IC'],
                                                                            test_perf['RIC'], test_perf['prec_10'], test_perf['sharpe5']), '\n\n')

print('Best Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_valid_perf['mse'], best_valid_perf['IC'], best_valid_perf['RIC'], best_valid_perf['prec_10'], best_valid_perf['sharpe5']))
print('Best Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_test_perf['mse'], best_test_perf['IC'], best_test_perf['RIC'], best_test_perf['prec_10'], best_test_perf['sharpe5']))

epoch1##########################################################
Valid : loss:2.51e-03  =  5.75e-04 + alpha*1.88e-02
Test: loss:2.06e-03  =  4.20e-04 + alpha*1.58e-02
Valid performance:
 mse:9.11e-04, IC:2.78e-02, RIC:2.51e-01, prec@10:5.33e-01, SR:2.83e+00
Test performance:
 mse:8.12e-04, IC:-5.31e-04, RIC:-8.75e-03, prec@10:5.21e-01, SR:-2.89e-01 


epoch2##########################################################
Valid : loss:2.52e-03  =  6.51e-04 + alpha*1.88e-02
Test: loss:2.07e-03  =  5.15e-04 + alpha*1.58e-02
Valid performance:
 mse:9.08e-04, IC:2.64e-02, RIC:2.44e-01, prec@10:5.29e-01, SR:2.45e+00
Test performance:
 mse:8.20e-04, IC:-1.20e-04, RIC:-1.93e-03, prec@10:5.24e-01, SR:-1.04e-01 


epoch3##########################################################
Valid : loss:2.51e-03  =  6.14e-04 + alpha*1.88e-02
Test: loss:2.06e-03  =  4.82e-04 + alpha*1.58e-02
Valid performance:
 mse:9.08e-04, IC:2.76e-02, RIC:2.50e-01, prec@10:5.30e-01, SR:2.92e+00
Test performance:
 mse:8.22e-04, I

##TST##

In [13]:
import random
import numpy as np
import os
import torch.nn as nn
import math
import torch
from load_data import load_EOD_data
from evaluator import evaluate
from model import get_loss
import pickle



np.random.seed(123456789)
torch.random.manual_seed(12345678)
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'

# Load data
data_path = '../dataset'
market_name = 'NASDAQ'
relation_name = 'wikidata'
stock_num = 1026
lookback_length = 16
epochs = 15
valid_index = 756
test_index = 1008
fea_num = 5
market_num = 20
steps = 1
learning_rate = 0.001
alpha = 0.1
scale_factor = 3
activation = 'GELU'

dataset_path = '../dataset/' + market_name
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/eod_data.pkl'), 'rb') as f:
        eod_data = pickle.load(f)
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/mask_data.pkl'), 'rb') as f:
    mask_data = pickle.load(f)
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/gt_data.pkl'), 'rb') as f:
    gt_data = pickle.load(f)
with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/price_data.pkl'), 'rb') as f:
    price_data = pickle.load(f)

trade_dates = mask_data.shape[1]

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term[:(d_model // 2)])
        
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerStockMixer(nn.Module):
    def __init__(self, input_dim, nhead, num_encoder_layers, dim_feedforward, dropout):
        super(TransformerStockMixer, self).__init__()
        self.model_type = 'Transformer'
        self.pos_encoder = PositionalEncoding(input_dim, dropout)
        encoder_layers = nn.TransformerEncoderLayer(input_dim, nhead, dim_feedforward, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_encoder_layers)
        self.encoder = nn.Linear(input_dim, input_dim)
        self.decoder = nn.Linear(input_dim, 1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src):
        src = self.encoder(src) * math.sqrt(src.size(2))
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output
    
def get_batch(offset=None):
    if offset is None:
        offset = random.randrange(0, valid_index)
    seq_len = lookback_length
    mask_batch = mask_data[:, offset: offset + seq_len + steps]
    mask_batch = np.min(mask_batch, axis=1)
    return (
        torch.Tensor(eod_data[:, offset:offset + seq_len, :]).to(device),
        torch.Tensor(np.expand_dims(mask_batch, axis=1)).to(device),
        torch.Tensor(np.expand_dims(price_data[:, offset + seq_len - 1], axis=1)).to(device),
        torch.Tensor(np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis=1)).to(device)
    )

def validate(model, start_index, end_index):
    model.eval()
    with torch.no_grad():
        cur_valid_pred = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_gt = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_mask = np.zeros([stock_num, end_index - start_index], dtype=float)
        loss = 0.
        reg_loss = 0.
        rank_loss = 0.
        for cur_offset in range(start_index - lookback_length - steps + 1, end_index - lookback_length - steps + 1):
            data_batch, mask_batch, price_batch, gt_batch = get_batch(cur_offset)
            prediction = model(data_batch).squeeze(-1)
            cur_loss, cur_reg_loss, cur_rank_loss, cur_rr = get_loss(prediction, gt_batch, price_batch, mask_batch,
                                                                     stock_num, alpha)
            loss += cur_loss.item()
            reg_loss += cur_reg_loss.item()
            rank_loss += cur_rank_loss.item()
            cur_valid_pred[:, cur_offset - (start_index - lookback_length - steps + 1)] = cur_rr[:, 0].cpu()
            cur_valid_gt[:, cur_offset - (start_index - lookback_length - steps + 1)] = gt_batch[:, 0].cpu()
            cur_valid_mask[:, cur_offset - (start_index - lookback_length - steps + 1)] = mask_batch[:, 0].cpu()
        loss = loss / (end_index - start_index)
        reg_loss = reg_loss / (end_index - start_index)
        rank_loss = rank_loss / (end_index - start_index)
        cur_valid_perf = evaluate(cur_valid_pred, cur_valid_gt, cur_valid_mask)
    return loss, reg_loss, rank_loss, cur_valid_perf

# Training
# Hyperparameters
input_dim = fea_num
nhead = 1
num_encoder_layers = 3
dim_feedforward = 128
dropout = 0.1

model = TransformerStockMixer(input_dim, nhead, num_encoder_layers, dim_feedforward, dropout).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_valid_loss = np.inf
best_valid_perf = None
best_test_perf = None
batch_offsets = np.arange(start=0, stop=valid_index, dtype=int)

for epoch in range(epochs):
    print("epoch{}##########################################################".format(epoch + 1))
    np.random.shuffle(batch_offsets)
    model.train()
    tra_loss = 0.0
    tra_reg_loss = 0.0
    tra_rank_loss = 0.0

    for j in range(valid_index - lookback_length - steps + 1):
        data_batch, mask_batch, price_batch, gt_batch = get_batch(batch_offsets[j])
        optimizer.zero_grad()
        prediction = model(data_batch).squeeze(-1)
        cur_loss, cur_reg_loss, cur_rank_loss, _ = get_loss(prediction, gt_batch, price_batch, mask_batch,
                                                            stock_num, alpha)
        cur_loss.backward()
        optimizer.step()

        tra_loss += cur_loss.item()
        tra_reg_loss += cur_reg_loss.item()
        tra_rank_loss += cur_rank_loss.item()
    tra_loss = tra_loss / (valid_index - lookback_length - steps + 1)
    tra_reg_loss = tra_reg_loss / (valid_index - lookback_length - steps + 1)
    tra_rank_loss = tra_rank_loss / (valid_index - lookback_length - steps + 1)
    print('Train : loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(tra_loss, tra_reg_loss, tra_rank_loss))

    val_loss, val_reg_loss, val_rank_loss, val_perf = validate(model, valid_index, test_index)
    print('Valid : loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(val_loss, val_reg_loss, val_rank_loss))

    test_loss, test_reg_loss, test_rank_loss, test_perf = validate(model, test_index, trade_dates)
    print('Test: loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(test_loss, test_reg_loss, test_rank_loss))

    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        best_valid_perf = val_perf
        best_test_perf = test_perf

    print('Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(val_perf['mse'], val_perf['IC'],
                                                         val_perf['RIC'], val_perf['prec_10'], val_perf['sharpe5']))
    print('Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(test_perf['mse'], test_perf['IC'],
                                                                        test_perf['RIC'], test_perf['prec_10'], test_perf['sharpe5']), '\n\n')
    
print('Best Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_valid_perf['mse'], best_valid_perf['IC'], best_valid_perf['RIC'], best_valid_perf['prec_10'], best_valid_perf['sharpe5']))
print('Best Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_test_perf['mse'], best_test_perf['IC'], best_test_perf['RIC'], best_test_perf['prec_10'], best_test_perf['sharpe5']))


c:\Users\asus\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


epoch1##########################################################
Train : loss:-2.02e+00  =  -3.94e+01 + alpha*2.46e+00
Valid : loss:-2.02e+00  =  -4.00e+01 + alpha*2.50e+00
Test: loss:-1.60e+00  =  -3.14e+01 + alpha*1.96e+00
Valid performance:
 mse:1.28e+01, IC:-2.48e-02, RIC:-2.39e-01, prec@10:5.30e-01, SR:-4.17e+00
Test performance:
 mse:1.29e+01, IC:-2.92e-03, RIC:-3.75e-02, prec@10:4.36e-01, SR:-8.94e+00 


epoch2##########################################################
Train : loss:-2.46e+00  =  -5.10e+01 + alpha*3.19e+00
Valid : loss:-2.08e+00  =  -4.15e+01 + alpha*2.59e+00
Test: loss:-1.73e+00  =  -3.35e+01 + alpha*2.09e+00
Valid performance:
 mse:2.00e+01, IC:-2.08e-02, RIC:-2.25e-01, prec@10:5.32e-01, SR:-3.47e+00
Test performance:
 mse:2.57e+01, IC:-5.77e-04, RIC:-8.05e-03, prec@10:4.59e-01, SR:-7.51e+00 


epoch3##########################################################
Train : loss:-2.53e+00  =  -5.32e+01 + alpha*3.33e+00
Valid : loss:-2.10e+00  =  -4.27e+01 + alpha*2.67e+

##STOCKMIXER MLP##

In [14]:
import random
import numpy as np
import os
import torch as torch
from load_data import load_EOD_data
from evaluator import evaluate
from model import get_loss, StockMixer
import pickle


np.random.seed(123456789)
torch.random.manual_seed(12345678)
device = torch.device("cuda") if torch.cuda.is_available() else 'cpu'

data_path = '../dataset'
market_name = 'NASDAQ'
relation_name = 'wikidata'
stock_num = 1026
lookback_length = 16
epochs = 100
valid_index = 756
test_index = 1008
fea_num = 5
market_num = 20
steps = 1
learning_rate = 0.001
alpha = 0.1
scale_factor = 3
activation = 'GELU'

dataset_path = '../dataset/' + market_name
if market_name == "SP500":
    data = np.load('../dataset/SP500/SP500.npy')
    data = data[:, 915:, :]
    price_data = data[:, :, -1]
    mask_data = np.ones((data.shape[0], data.shape[1]))
    eod_data = data
    gt_data = np.zeros((data.shape[0], data.shape[1]))
    for ticket in range(0, data.shape[0]):
        for row in range(1, data.shape[1]):
            gt_data[ticket][row] = (data[ticket][row][-1] - data[ticket][row - steps][-1]) / \
                                   data[ticket][row - steps][-1]
else:
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/eod_data.pkl'), 'rb') as f:
        eod_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/mask_data.pkl'), 'rb') as f:
        mask_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/gt_data.pkl'), 'rb') as f:
        gt_data = pickle.load(f)
    with open(os.path.join(dataset_path, 'D:/Kuliah/Semester_6/Business Intelligence/Code/Business_Intelligence_project/dataset/NASDAQ/price_data.pkl'), 'rb') as f:
        price_data = pickle.load(f)

trade_dates = mask_data.shape[1]
model = StockMixer(
    stocks=stock_num,
    time_steps=lookback_length,
    channels=fea_num,
    market=market_num,
    scale=scale_factor
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
best_valid_loss = np.inf
best_valid_perf = None
best_test_perf = None
batch_offsets = np.arange(start=0, stop=valid_index, dtype=int)


def validate(start_index, end_index):
    with torch.no_grad():
        cur_valid_pred = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_gt = np.zeros([stock_num, end_index - start_index], dtype=float)
        cur_valid_mask = np.zeros([stock_num, end_index - start_index], dtype=float)
        loss = 0.
        reg_loss = 0.
        rank_loss = 0.
        for cur_offset in range(start_index - lookback_length - steps + 1, end_index - lookback_length - steps + 1):
            data_batch, mask_batch, price_batch, gt_batch = map(

                lambda x: torch.Tensor(x).to(device),
                get_batch(cur_offset)
            )
            prediction = model(data_batch)
            cur_loss, cur_reg_loss, cur_rank_loss, cur_rr = get_loss(prediction, gt_batch, price_batch, mask_batch,
                                                                     stock_num, alpha)
            loss += cur_loss.item()
            reg_loss += cur_reg_loss.item()
            rank_loss += cur_rank_loss.item()
            cur_valid_pred[:, cur_offset - (start_index - lookback_length - steps + 1)] = cur_rr[:, 0].cpu()
            cur_valid_gt[:, cur_offset - (start_index - lookback_length - steps + 1)] = gt_batch[:, 0].cpu()
            cur_valid_mask[:, cur_offset - (start_index - lookback_length - steps + 1)] = mask_batch[:, 0].cpu()
        loss = loss / (end_index - start_index)
        reg_loss = reg_loss / (end_index - start_index)
        rank_loss = rank_loss / (end_index - start_index)
        cur_valid_perf = evaluate(cur_valid_pred, cur_valid_gt, cur_valid_mask)
    return loss, reg_loss, rank_loss, cur_valid_perf


def get_batch(offset=None):
    if offset is None:
        offset = random.randrange(0, valid_index)
    seq_len = lookback_length
    mask_batch = mask_data[:, offset: offset + seq_len + steps]
    mask_batch = np.min(mask_batch, axis=1)
    return (
        eod_data[:, offset:offset + seq_len, :],
        np.expand_dims(mask_batch, axis=1),
        np.expand_dims(price_data[:, offset + seq_len - 1], axis=1),
        np.expand_dims(gt_data[:, offset + seq_len + steps - 1], axis=1))


for epoch in range(epochs):
    print("epoch{}##########################################################".format(epoch + 1))
    np.random.shuffle(batch_offsets)
    tra_loss = 0.0
    tra_reg_loss = 0.0
    tra_rank_loss = 0.0
    for j in range(valid_index - lookback_length - steps + 1):
        data_batch, mask_batch, price_batch, gt_batch = map(
            lambda x: torch.Tensor(x).to(device),
            get_batch(batch_offsets[j])
        )
        optimizer.zero_grad()
        prediction = model(data_batch)
        cur_loss, cur_reg_loss, cur_rank_loss, _ = get_loss(prediction, gt_batch, price_batch, mask_batch,
                                                            stock_num, alpha)
        cur_loss = cur_loss
        cur_loss.backward()
        optimizer.step()

        tra_loss += cur_loss.item()
        tra_reg_loss += cur_reg_loss.item()
        tra_rank_loss += cur_rank_loss.item()
    tra_loss = tra_loss / (valid_index - lookback_length - steps + 1)
    tra_reg_loss = tra_reg_loss / (valid_index - lookback_length - steps + 1)
    tra_rank_loss = tra_rank_loss / (valid_index - lookback_length - steps + 1)
    print('Train : loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(tra_loss, tra_reg_loss, tra_rank_loss))

    val_loss, val_reg_loss, val_rank_loss, val_perf = validate(valid_index, test_index)
    print('Valid : loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(val_loss, val_reg_loss, val_rank_loss))

    test_loss, test_reg_loss, test_rank_loss, test_perf = validate(test_index, trade_dates)
    print('Test: loss:{:.2e}  =  {:.2e} + alpha*{:.2e}'.format(test_loss, test_reg_loss, test_rank_loss))

    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        best_valid_perf = val_perf
        best_test_perf = test_perf

    print('Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(val_perf['mse'], val_perf['IC'],
                                                     val_perf['RIC'], val_perf['prec_10'], val_perf['sharpe5']))
    print('Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(test_perf['mse'], test_perf['IC'],
                                                                            test_perf['RIC'], test_perf['prec_10'], test_perf['sharpe5']), '\n\n')
    
print('Best Valid performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_valid_perf['mse'], best_valid_perf['IC'], best_valid_perf['RIC'], best_valid_perf['prec_10'], best_valid_perf['sharpe5']))
print('Best Test performance:\n', 'mse:{:.2e}, IC:{:.2e}, RIC:{:.2e}, prec@10:{:.2e}, SR:{:.2e}'.format(
    best_test_perf['mse'], best_test_perf['IC'], best_test_perf['RIC'], best_test_perf['prec_10'], best_test_perf['sharpe5']))


epoch1##########################################################
Train : loss:3.63e-03  =  -4.43e-02 + alpha*5.38e-02
Valid : loss:1.80e-03  =  -1.20e-02 + alpha*2.29e-02
Test: loss:1.79e-03  =  -7.25e-03 + alpha*1.98e-02
Valid performance:
 mse:1.01e-03, IC:-1.37e-02, RIC:-1.53e-01, prec@10:5.21e-01, SR:9.90e-01
Test performance:
 mse:8.80e-04, IC:-1.15e-02, RIC:-1.51e-01, prec@10:5.35e-01, SR:4.87e-01 


epoch2##########################################################
Train : loss:1.52e-03  =  -3.54e-02 + alpha*3.98e-02
Valid : loss:1.33e-03  =  -2.05e-02 + alpha*2.62e-02
Test: loss:1.24e-03  =  -1.53e-02 + alpha*2.20e-02
Valid performance:
 mse:1.23e-03, IC:-1.43e-02, RIC:-1.62e-01, prec@10:5.24e-01, SR:4.51e-01
Test performance:
 mse:1.01e-03, IC:-9.41e-03, RIC:-1.32e-01, prec@10:5.25e-01, SR:-3.11e-01 


epoch3##########################################################
Train : loss:1.27e-03  =  -3.15e-02 + alpha*3.51e-02
Valid : loss:1.25e-03  =  -2.16e-02 + alpha*2.65e-02
Test: lo